<img align="center" src="../extra/logos/logos.png" width='800px'>

# Operaciones con el DataCube

En ese apartado, se verán las operaciones básicas que se pueden realizar con el DataCube:
1. Definir una área de estudio
1. Buscar productos y mediciones
1. Solicitar la información
1. Visualizar la información

Se puede tener una idea de los productos y la cobertura de las imágenes disponibles en el cubo en el [Explorador de imágenes](https://explorer.stage.earth.dataobservatory.net/).


>Nota: Este notebook contiene elementos extraídos desde [DataCube Australia](https://github.com/GeoscienceAustralia/dea-notebooks).

## 1. Products / Measurements

En primera instancia, es necesario definir qué son cada uno:
1. Products: hace referencia al sensor que obtiene la información/imagen (Landsat 5, Sentinel-2, etc).
1. Measurements: varían de producto en producto y hacen referencia a las bandas/mediciones disponibles para cada sensor. En algunos casos, también incluye las bandas de calidad y otros sub-productos útiles.

Primero, es necesario cargar los paquetes que permiten acceder al DataCube y sus funcionalidades:

In [1]:
import datacube
import pandas as pd
from odc.ui import DcViewer

from datacube.utils.rio import configure_s3_access
configure_s3_access(aws_unsigned=False, requester_pays=True)

Es necesario generar una app del cubo con un nombre. Esto no afecta en nada el desemepño y es solamente por motivos de trazabilidad en caso de algún problema. El nombre puede ser de cualquier tipo, pero se recomienda que sea algo relacionado con la aplicación que se le dará.

In [2]:
dc = datacube.Datacube(app='01_operaciones')

### 1.1 Products

Una vez se tiene la app funcionando (en este caso `dc`), se pueden listar los productos disponibles.

In [3]:
dc.list_products()

,name,description,product_family,dataset_maturity,creation_time,lat,platform,lon,region_code,label,instrument,format,time,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,,,
8,fasat_charlie,FASat Charlie data,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
15,s2_l2a,"Sentinel-2a and Sentinel-2b imagery, processed...",None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
10,srtm,1 arc second elevation model,None,None,None,None,None,None,None,None,None,None,None,EPSG:4326,"(-0.0002777777777777778, 0.0002777777777777778)",NaN,"(latitude, longitude)"
9,ultracam_saf,LEARJET con ULTRACAM (SAF),None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
13,usgs_aws_ls5c2_sr,Landsat 5 USGS Collection 2 Surface Reflectanc...,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
12,usgs_aws_ls7c2_sr,Landsat 7 USGS Collection 2 Surface Reflectanc...,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
14,usgs_aws_ls8c2_sr,Landsat 8 USGS Collection 2 Surface Reflectanc...,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
1,usgs_espa_ls5c1_sr,Landsat 5 USGS Collection 1 Surface Reflectanc...,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
2,usgs_espa_ls7c1_sr,Landsat 7 USGS Collection 1 Surface Reflectanc...,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN


Actualmente está disponible la serie completa de Landsat (5, 7 y 8), Sentinel-2 y el DEM SRTM. Como se puede apreciar en el listado, existe más de una definición para la serie Landsat, cuya principal diferencia es si están en un bucket propio del DO o son parte de buckets públicos de AWS. Se recomienda utilizar las fuentes provenientes de este último.

Para poder diferenciarlos, en el caso de Landsat el nombre contiene `aws` (en la columna `name` de la tabla anterior); por ejemplo, usgs_aws_l8c2_sr, usgs_aws_l7c2_sr y usgs_aws_l5c2_sr. Esta serie cubre todo Chile y está actualizada a la fecha más reciente. Por otro lado, aquellas imágenes Landsat que están el bucket propio, contienen `espa` en su nombre (usgs_espa_ls8c1_sr).

### 1.2 Measurements

Cada producto, contiene mediciones (`measurements`) a las cuáles es posible acceder. A continuación se muestra un listado de las mediciones disponibles para dos sensores (Landsat 8 y Sentinel-2)

In [4]:
products = ['usgs_aws_ls8c2_sr', 's2_l2a']
dc.list_measurements().loc[products]

name  \
product           measurement                                                      
usgs_aws_ls8c2_sr coastal_aerosol                                coastal_aerosol   
                  blue                                                      blue   
                  green                                                    green   
                  red                                                        red   
                  nir                                                        nir   
                  swir1                                                    swir1   
                  swir2                                                    swir2   
                  surface_temperature                        surface_temperature   
                  thermal_radiance                              thermal_radiance   
                  upwell_radiance                                upwell_radiance   
                  downwell_radiance                            downwell_radiance   
                  atmospheric_transmittance            atmospheric_transmittance   
                  emissivity                                          emissivity   
                  emissivity_stdev                              emissivity_stdev   
                  cloud_distance                                  cloud_distance   
                  quality_l2_surface_temperature  quality_l2_surface_temperature   
                  pixel_qa                                              pixel_qa   
                  aerosol_qa                                          aerosol_qa   
                  radsat_qa                                            radsat_qa   
s2_l2a            B01                                                        B01   
                  B02                                                        B02   
                  B03                                                        B03   
                  B04                                                        B04   
                  B05                                                        B05   
                  B06                                                        B06   
                  B07                                                        B07   
                  B08                                                        B08   
                  B8A                                                        B8A   
                  B09                                                        B09   
                  B11                                                        B11   
                  B12                                                        B12   
                  SCL                                                        SCL   
                  AOT                                                        AOT   
                  WVP                                                        WVP   

                                                   dtype        units  nodata  \
product           measurement                                                   
usgs_aws_ls8c2_sr coastal_aerosol                 uint16  reflectance     0.0   
                  blue                            uint16  reflectance     0.0   
                  green                           uint16  reflectance     0.0   
                  red                             uint16  reflectance     0.0   
                  nir                             uint16  reflectance     0.0   
                  swir1                           uint16  reflectance     0.0   
                  swir2                           uint16  reflectance     0.0   
                  surface_temperature             uint16       Kelvin     0.0   
                  thermal_radiance                uint16            1 -9999.0   
                  upwell_radiance                 uint16            1 -9999.0   
                  downwell_radiance               uint16            1 -9999.0  

Cada una de estas filas se corresponde con una "banda" del sensor. Como se puede apreciar, no todas corresponden a mediciones directas realizadas por el sensor. En el caso de Landast y Sentinel, siempre están presentes bandas de calidad. También hay algunos sub-productos en algunos casos.

Hay que prestar atención a las columnas `dtype`, `units` y `nodata`. En ellas se indican el tipo de dato (entero corto, largo, flotante, etc), las unidades y el valor de datos NA. Hay que tenerlos presente al momento de trabajar con alguna de estas métricas. Por ejemplo, las bandas de reflectancia poseen tipo entero (uint16), pero la reflectancia posee valores en el rango `[0-1]`; es por esta razón que es necesario escalar la información (10.000 en este caso).

## 2. Cargar información

Para poder cargar las imágenes deseadas, es necesario definir algunos puntos primero:

1. Definir producto
1. Definir ventana espacial de trabajo (latitud, longitud).
1. Definir ventana temporal de trabajo.
1. Definir una proyección de salida.
1. Definir una resolución de salida.

Y la forma de cargar el requerimiento en el cubo:

In [5]:
ds = dc.load(product=products[0],
             x=(-72, -70),
             y=(-33, -32),
             time=("2020-01-01", "2021-12-31"),
             output_crs='EPSG:32719',
             resolution=(-30, 30),
             dask_chunks={"time": 1, 'x':2048, 'y':2048})

Antes de mirar el objeto, hay que destacar que se puede solicitar la información en una resolución diferente a la nativa. Para ello, basta con modificar la opción de resolución a la deseada. Por defecto, se utiliza el método de remuestreo "nearest neighbour", pero los siguientes están también disponibles:
```
"nearest", "cubic", "bilinear", "cubic_spline", "lanczos", 
"average", "mode", "gauss", "max", "min", "med", "q1", "q3"
```

>**Nota**: Las unidades de la opción `resolution` corresponden a las unidades del crs específicado en `output_crs`. Hay que prestar atención a que normalmente el primer elemento es negativo, dada la forma en que se almacenan las coordenadas en las imágenes (el inicio está esquina superior izquierda) y la forma en que operan la mayoría de los crs (con un "inicio" en la esquina inferior izquierda).

Ahora, un vistazo al objeto resultado:

In [6]:
ds

<xarray.Dataset>
Dimensions:                         (time: 144, x: 6334, y: 3815)
Coordinates:
  * time                            (time) datetime64[ns] 2020-01-01T14:39:40...
  * y                               (y) float64 6.459e+06 ... 6.345e+06
  * x                               (x) float64 2.166e+05 ... 4.066e+05
    spatial_ref                     int32 32719
Data variables:
    coastal_aerosol                 (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    blue                            (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green                           (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red                             (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir                             (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir1                           (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    swir2                           (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    surface_temperature             (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    thermal_radiance                (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    upwell_radiance                 (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    downwell_radiance               (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    atmospheric_transmittance       (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    emissivity                      (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    emissivity_stdev                (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    cloud_distance                  (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    quality_l2_surface_temperature  (time, y, x) int16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    pixel_qa                        (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    aerosol_qa                      (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    radsat_qa                       (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32719
    grid_mapping:  spatial_ref

De esta salida, hay un par de cosas a las que prestar atención. 

Primero, el tipo de objeto resultante. `OpenDataCube` funciona sobre la librería [`xarray`](https://xarray-contrib.github.io/xarray-tutorial/scipy-tutorial/00_overview.html), la cual permite trabajar en multiples dimensiones de manera eficiente. La estructura de datos es muy similar al formato [NetCDF](https://psl.noaa.gov/data/gridded/whatsnetCDF.html) y una de las principales ventajas, es que permite trabajar con la librería [`Dask`](https://dask.org/), la cual permite paralelizar cargas de trabajo.

1. ***Dimensions***: es un listado simple, que identifica las dimensiones del arreglo. En este caso es tridimencinoal con (tiempo, latitud, longitud), pero cualquier otra combinación es posbible (1, 2, 3, 4,...., X dimensiones).
1. ***Coordinates***: identifica el valor de cada una de las dimensiones, en cada uno de sus "instantes". En este caso `time` representa la fecha para cada tiempo y `x` e `y` son las coordenadas geográficas de cada pixel.
1. ***Data variables***: son las mediciones, normalmente "bandas" del sensor, pero puede ser cualquier otro tipo. En este caso, por cada unidad `time`, existe una imagen completa con coordenadas `x`, `y` (un objeto `xarray.DataArray`).
1. ***Attributes***: identifica el sistema de coordenadas de referencia de la información cargada.

Se puede ver con mayor detalle alguna de las variables/mediciones. En el fondo esta una colección dentro de otra.

In [7]:
ds['nir'] # es equivalente a ds.nir

<xarray.DataArray 'nir' (time: 144, y: 3815, x: 6334)>
dask.array<dc_load_nir, shape=(144, 3815, 6334), dtype=uint16, chunksize=(1, 2048, 2048), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2020-01-01T14:39:40.409657 ... 2021-04...
  * y            (y) float64 6.459e+06 6.459e+06 ... 6.345e+06 6.345e+06
  * x            (x) float64 2.166e+05 2.166e+05 ... 4.065e+05 4.066e+05
    spatial_ref  int32 32719
Attributes:
    units:         reflectance
    nodata:        0
    crs:           EPSG:32719
    grid_mapping:  spatial_ref

Del objeto anterior, es importante resaltar algunos aspectos

1. Bytes: Tamaño total de la colección, y el tamaño de cada chunk.
1. Shape: Dimensiones del arreglo. En este caso es (tiempo, y, x) para toda la colección solicitada y para cada chunk. En este último caso, un chunk sólo abarca un tiempo.
1. Count: Número de tareas pendientes y número de chunks. A mayor tamaño del chunk, menor número de tareas a calcular.
1. Type: tipo de datos (entero, flotante, etc) y estructura (numpy array, dask array, etc).

>**Nota**: El tamaño "ideal" del chunk, debiera rondar los 100 MB.

Por lo demás, las coordenadas y atributos son similares al objeto anterior.

Un punto importante sobre el funcionamiento del DataCube, es que `xarray` es *lazy*, lo que quiere decir que no hace todo los cálculos en el momento en que se escribe, si no que lo hace sólo cuando es necesario. Es importante tener esto presente, porque por cada acción que se le solicita al cubo, se van acumulando tareas y si se llega a un número demasiado alto, al momento de hacer un cálculo tomará demasiado tiempo y el sistema podría colapsar.

Cuando se llega a un punto en que se sabe que se sabe será utilizado como base para otros (en el fondo un checkpoint), es posible pedirle al cubo que resuelva todas las tareas pendientes por medio de `ds.persist()`. Más adelante se verán algunos ejemplos, pero es importante tenerlo persente.

>**Nota**: ¿Cuándo invocar `persist()`? No hay una regla definitiva, pero normalmente, cuando se han acumulado alrededor de 50.000 Tasks. También es bueno invocarlo antes de por ejemplo pedir varias métricas diferentes sobre el mismo arreglo, caso contrario, cada vez que se pide una métrica se debe calcular todo cada vez (esto incluye la generación de imágenes).

## 3. Visualizaciones

In [11]:
%matplotlib inline

import sys
sys.path.append("../../dea-notebooks/Scripts")

from dea_plotting import display_map
from dea_plotting import rgb

In [13]:
display_map(x=(-72, -70), y=(-33, -32))

## 4. Exportar información

1. tif
2. netcdf

# Mini - Tarea

Repita este notebook, pero utilice una zona/sensor diferente al de los ejemplos (no utilice una zona muy extensa o una ventana temporal demasiado amplia).

1. Explorar diferentes ventanas espacio-temporales.
1. Explorar productos.
1. Probar diferentes configuraciones de consulta.